In [ ]:
import pandas as pd
import openpyxl

wb = openpyxl.load_workbook('./data/HHD_C_Report_2023Q4.xlsx',
                           data_only=True)
dfs = {
    x: pd.DataFrame(wb[x].values) for x in wb.sheetnames
    if 'Data' in x
}
dfs['Page 3 Data']
credit_df = dfs['Page 3 Data'].iloc[3:, :8]
names = credit_df.iloc[0, :].to_list()
names[0] = 'Yr:Q'
credit_df.columns = names
credit_df = credit_df.iloc[1:-5, :].set_index('Yr:Q', drop=True)
print(credit_df.to_string())

In [ ]:
ax = credit_df.reset_index().iloc[:, :-1].plot(
    x='Yr:Q', kind='bar', stacked=True, figsize=(11,8),
    fontsize=6.0
)
_ = ax.set_xticks(ax.get_xticks(), labels=ax.get_xticklabels(), rotation=60, ha='right')

In [ ]:
total_debt_df = credit_df.loc[:, 'Total'].to_frame()
total_debt_df.info()
total_debt_df['Delta'] = total_debt_df['Total'].pct_change()
print(total_debt_df.to_string())
print(f'Num. decreases = {total_debt_df.query("Delta < 0").shape[0]}')
print(f'Num. increases = {total_debt_df.query("Delta > 0").shape[0]}')

In [ ]:
dfp10 = dfs['Page 10 Data']
#print(dfp10)
colnames = 'YYQ', 'CCB', 'CCAB', 'CCL', 'HERB', 'HERAB', 'HERL'
dfp10 = (
    dfp10.iloc[4:, :7].rename(columns={x: colnames[x] for x in range(7)})
         #.set_index('YYQ', drop=True)
)
dfp10.iloc[0:-1:2,4:] = dfp10.iloc[1:-1:2, 4:]
dfp10 = dfp10.iloc[0:-1:2, :]
dfp10x = pd.MultiIndex.from_tuples(
    dfp10['YYQ'].apply(lambda x: (int(x[:2]) + 2000, int(x[-1]))),
    names=['YY', 'Q']
)
dfp10 = dfp10.set_index(dfp10x).drop('YYQ', axis=1)
#dfp10['YY'] = dfp10['YYQ'].apply(lambda x: 2000 + int(x[:2]))
#dfp10['Q'] = dfp10['YYQ'].apply(lambda x: int(x[-1]))
#dfp10 = dfp10.iloc[:, 1:].set_index(['YY', 'Q'], drop=True)
print(dfp10.iloc[:8, :].to_string())
dfp10.groupby(level=0).mean().loc[:, ['CCB', 'CCL']].plot()

In [ ]:
dfp12 = dfs['Page 12 Data']
#print(dfp12.iloc[:,:4].to_string())
colnames = ['YYQ'] + dfp12.iloc[3, 1:].to_list()
#print(colnames)
delinq_df = (
    dfp12.iloc[4:, :7].rename(columns={x: colnames[x] for x in range(8)})
      .set_index(
        pd.MultiIndex.from_tuples(
          dfp12.iloc[4:, 0].apply(
            lambda x: (2000 + int(x[:2]), int(x[-1]))),
            names=['YY', 'Q']
          )
       ).drop('YYQ', axis=1)
)
delinq_df

In [ ]:
dfs['Page 32 Data']
